<a href="https://colab.research.google.com/github/MaxAkkers/DHANS/blob/main/ModelTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 6.7MB/s 
     |████████████████████████████████| 901kB 49.3MB/s 
     |████████████████████████████████| 3.3MB 49.0MB/s 


In [1]:
! git clone https://github.com/gijswijnholds/sick_nl.git

Cloning into 'sick_nl'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 318 (delta 166), reused 177 (delta 71), pack-reused 0
Receiving objects: 100% (318/318), 697.77 KiB | 4.71 MiB/s, done.
Resolving deltas: 100% (166/166), done.


In [7]:
!pip install -r sick_nl/requirements.txt

In [12]:
import os

In [13]:
os.getcwd()

'/content/gdrive/MyDrive/sick_nl_working'

In [14]:
from sick_nl.code.evaluation.eval_nli import evaluate_en_nli_models, evaluate_nl_nli_models, run_finetuner
from sick_nl.code.loaders.sick import load_sick_nl
from sick_nl.code.config import models_folder, results_folder, bert, bert_nl, roberta, roberta_nl, mbert

In [32]:

bert = "bert-base-cased"
bert_nl = "wietsedv/bert-base-dutch-cased"
roberta = "roberta-base"
roberta_nl = "pdelobelle/robbert-v2-dutch-base"
mbert = "bert-base-multilingual-cased"

data_folder = 'sick_nl/data'
stress_test_folder = os.path.join(data_folder, 'tasks/stress_tests')
sick_folder = os.path.join(data_folder, 'tasks/sick_nl')
vectors_folder = os.path.join(data_folder, 'vectors')

sick_fn_en = os.path.join(sick_folder, 'SICK.txt')
sick_fn_nl = os.path.join(sick_folder, 'heuristics_set.txt')

skipgram_fn = os.path.join(vectors_folder, 'GoogleNews-vectors-negative300_SICK.txt')
skipgram_fn_nl = os.path.join(vectors_folder, '320/wikipedia-320.txt')

model_data_folder = 'sick_nl/model_data'
models_folder = os.path.join(model_data_folder, 'models')
models_agg_folder = os.path.join(model_data_folder, 'models_agg')
results_folder = os.path.join(model_data_folder, 'results')

prep_order_fn = os.path.join(stress_test_folder, 'prep_phrase_order.txt')
present_tense_fn = os.path.join(stress_test_folder, 'present_cont_present_simple.txt')

best_bert_nl_model = os.path.join(models_agg_folder, 'model_SICK_NL_bert-base-dutch-cased_epoch16.pt')
best_roberta_nl_model = os.path.join(models_agg_folder, 'model_SICK_NL_robbert-v2-dutch-base_epoch19.pt')
best_mbert_nl_model = os.path.join(models_agg_folder, 'model_SICK_NL_bert-base-multilingual-cased_epoch16.pt')

In [33]:
nl_sick = load_sick_nl()

[['Als de leerlingen leren , verlaten de jongens ', 'De leerlingen leren ', 'NEUTRAL', 'constituent', 'TEST'], ['Als de studenten ontwijken , studeren de vrouwen ', 'De studenten ontwijken ', 'NEUTRAL', 'constituent', 'TEST'], ['Als de klimmers knuffelen , redden de spelers ', 'De klimmers knuffelen ', 'NEUTRAL', 'constituent', 'TEST'], ['Als de leerlingen voeren , duelleren de dochters ', 'De leerlingen voeren ', 'NEUTRAL', 'constituent', 'TEST'], ['Als de jongens leren , duelleren de soldaten ', 'De jongens leren ', 'NEUTRAL', 'constituent', 'TEST'], ['Als de spelers redden , voeren de fotografen ', 'De spelers redden ', 'NEUTRAL', 'constituent', 'TEST'], ['Als de jongens trekken , telefoneren de klimmers ', 'De jongens trekken ', 'NEUTRAL', 'constituent', 'TEST'], ['Als de mimespelers worstelen , tekenen de fotografen ', 'De mimespelers worstelen ', 'NEUTRAL', 'constituent', 'TEST'], ["Als de studenten gooien , worstelen de boa's ", 'De studenten gooien ', 'NEUTRAL', 'constituent', 

In [ ]:
print(f"{models_folder}\n{results_folder}\n{bert}\n{bert_nl}\n{roberta}\n{roberta_nl}\n{mbert}")

sick_nl/model_data/models
sick_nl/model_data/results
bert-base-cased
wietsedv/bert-base-dutch-cased
roberta-base
pdelobelle/robbert-v2-dutch-base
bert-base-multilingual-cased


In [ ]:
nl_sick

In [ ]:
run_finetuner(nl_sick, bert_nl, setting='bert', num_epochs=20, model_folder=models_folder, result_folder=results_folder)
run_finetuner(nl_sick, roberta_nl, setting='roberta', num_epochs=20, model_folder=models_folder, result_folder=results_folder)
run_finetuner(nl_sick, mbert, setting='bert', num_epochs=20, model_folder=models_folder, result_folder=results_folder)

Loading BERT model...


KeyboardInterrupt: ignored

In [17]:
import torch
from sick_nl.code.loaders.nli_models import load_bert_nli_model
from sick_nl.code.models.bert_finetune import SICK_BERT_DATASET, BERTFineTuner
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [18]:
! grep -R "torch.load" sick_nl/code

sick_nl/code/evaluation/eval_nli.py:    best_model = torch.load(best_model_fn)
sick_nl/code/loaders/nli_models.py:    model = torch.load(model_fn)


In [31]:
class SICK(object):
    def __init__(self, sick_fn: str):
        self.sick_fn = sick_fn
        self.name = "bert"
        self.data = self.load_data()
        self.train_data, self.dev_data, self.test_data = self.split_data()

    def load_data(self):
        with open(self.sick_fn, 'r') as in_file:
            lines = [ln.strip().split('\t') for ln in in_file.readlines()][1:]
        sentence_data = [(s1, s2, el, rl, split)
                         for (s1, s2, el, rl, split) in lines]
        print(sentence_data)
        return sentence_data

    def split_data(self):
        train_data, dev_data, test_data = [], [], []
        for (s1, s2, el, rl, s) in self.data:
            if s == 'TRAIN':
                train_data.append((s1, s2, el, rl))
            if s == 'TRIAL':
                dev_data.append((s1, s2, el, rl))
            if s == 'TEST':
                test_data.append((s1, s2, el, rl))
        return train_data, dev_data, test_data

def load_sick_en():
    return SICK(sick_fn_en)

def load_sick_nl():
    sick_fn_nl = 'sick_nl/data/tasks/sick_nl/heuristics_set.txt'
    return SICK(sick_fn_nl)

In [21]:
from sick_nl.code.evaluation.eval_nli  import consolidate_results_and_models

def only_evaluate(sick_dataset, model_fn, name, setting='bert'):
    #tuner = load_bert_nli_model(sick_dataset, name, setting)
    print("Loading BERT model...")
    if setting == 'bert':
        tokenizer = BertTokenizer.from_pretrained(name)
        model = torch.load(model_fn)
    elif setting == 'roberta':
        tokenizer = RobertaTokenizer.from_pretrained(name)
        model = torch.load(model_fn)

    print("Preparing datasets...")
    train_dataset = SICK_BERT_DATASET(sick_dataset.train_data, tokenizer)
    eval_dataset = SICK_BERT_DATASET(sick_dataset.test_data, tokenizer)
    print("Loading finetuning model...")
    tuner = BERTFineTuner(name, tokenizer, model, train_dataset, eval_dataset,
                         num_epochs=1, freeze=False)
    print(f"Accuracy is {tuner.evaluate()}!")
    #consolidate_results_and_models(sick_dataset.name, name, model_folder, result_folder)

In [34]:
fine_tuned_model = 'sick_nl/model_data/models_agg/model_SICK_NL_bert-base-dutch-cased_epoch12.pt'
only_evaluate(nl_sick, fine_tuned_model, "wietsedv/bert-base-dutch-cased", setting='bert')
fine_tuned_model = 'sick_nl/model_data/models_agg/model_SICK_NL_robbert-v2-dutch-base_epoch1.pt'
only_evaluate(nl_sick, fine_tuned_model, "pdelobelle/robbert-v2-dutch-base", setting='roberta')
fine_tuned_model = 'sick_nl/model_data/models_agg/model_SICK_NL_robbert-v2-dutch-base_epoch1.pt'
only_evaluate(nl_sick, fine_tuned_model, "bert-base-multilingual-cased", setting='bert')


Loading BERT model...


loading file https://huggingface.co/wietsedv/bert-base-dutch-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/67c36a7a6f549d5b7c7f0c0eb4b0379f55015a54c5ab1a88909e12f900553b8b.2bab3c86db6bbfc2b6fe96a4a416e86c3d3a408fe0fb2eb53d5deb7d671280b9
loading file https://huggingface.co/wietsedv/bert-base-dutch-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/wietsedv/bert-base-dutch-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/wietsedv/bert-base-dutch-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c6a1665d5ce2318e4defa6b1d6cc5a5ce12ad9f9861a3c6d28e76cfdc9727c75.c9201be1323c49787de3aab70bd869fbc376d46a1c44c7a0075d8e97358ab103
loading file https://huggingface.co/wietsedv/bert-base-dutch-cased/resolve/main/tokenizer.json from cache at None
PyTorch: setting up devices
The default value for the training argument `--report_to` will

Preparing datasets...
Loading finetuning model...


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]" of type <class 'numpy.ndarray'> for key "eval/trues" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped thi

Accuracy is {'eval_loss': 1.2267309427261353, 'eval_accuracy': 0.1285140562248996, 'eval_f1': 0.11387900355871884, 'eval_precision': 0.5, 'eval_recall': 0.0642570281124498, 'eval_trues': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 

loading file https://huggingface.co/pdelobelle/robbert-v2-dutch-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/e338cb20bd645675d888a6469d1b7bfe472e69a5c93052006528c9ae4f856e44.43004df1e5c2251acdd15f077a74063dea7f0082e895ca8978bc6876d97c918b
loading file https://huggingface.co/pdelobelle/robbert-v2-dutch-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/0cc0d13f8f47c5f67868f6ea3638b66b788e833af7323164ba7d3e1c92f49f42.15e46d82fe3fef52038bdadabeb8cca392378e0966fa780352e52eaa8301f2ba
loading file https://huggingface.co/pdelobelle/robbert-v2-dutch-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/pdelobelle/robbert-v2-dutch-base/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/fc901152bccc6123aa5ca59de0c36b6e68e0796b4d7875a0e4e9744aab712cd2.a11ebb04664c067c8fe5ef8f8068b0f721263414a26058692f7b2e4ba2a1b342
loading file https://huggingface.co/pdelobe

Preparing datasets...
Loading finetuning model...


Trainer is attempting to log a value of "[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]" of type <class 'numpy.ndarray'> for key "eval/trues" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2

Accuracy is {'eval_loss': 1.3978139162063599, 'eval_accuracy': 0.020080321285140562, 'eval_f1': 0.01968503937007874, 'eval_precision': 0.5, 'eval_recall': 0.010040160642570281, 'eval_trues': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
      